# Understanding GPU/Vectorized Computing
- Numba is an open-source just-in-time (JIT) compiler
    - Translates optimized machine code using LLVM
    - You can specify which funciton sand blocks of code you want to be compiled at runtime
    - JIT has no guarantee of faster code; overhead to compile, subsequent calls are faster!
    - Common Numba decorators
        - @jit -> lazy compilation: second function call uses the already optimized machine code
            - General purpose decorator that optimizes different functions
        - @vectorize -> eager compilation: compiled at decoration
            - Allows functions to operate on arrays as if they are using scalars
    - Use the nopython=True option to leverage primitive-optimized code that doesn't need the Python interpreter
- f-string formatting
    - g causes (among other things) insignificant trailing zeros [to be] removed from the significand, and the decimal point is also removed if there are no remaining digits following it.

In [1]:
import math
import numpy as np

from numpy.random import default_rng
from time import time
from numba import jit, vectorize, double, int64, uint8, boolean

In [2]:
def timeit(func):
    def inner(*args, **kwargs):
        start_time = time()
        result = func(*args, **kwargs)
        timed = time() - start_time
        fractional_portion = str(timed).split('.')[1]
        length = len(fractional_portion)
        
        ptr = 0
        count = 0
        
        while ptr < length:
            if fractional_portion[ptr] == '0':
                count += 1
                ptr += 1
            else:
                ptr = length
                
        if count == 1:
            print('It was so fast that we couldn\'t record it. Time goes to the 18th fractional point (exa)')
        elif count <= 3:
            print(f'Timed at {timed * 10**3:g} milliseconds')
        elif count <= 6:
            print(f'Timed at {timed * 10**6:g} microseconds')
        elif count <= 9:
            print(f'Timed at {timed * 10**9:g} nanoseconds')
        elif count <= 12:
            print(f'Timed at {timed * 10**12:g} picoseconds')
        else:
            print(f'Time at {timed * 10**15:g} attosecons')
        
        return result
    return inner

In [3]:
@timeit
@jit
def euclidean_distance_pt(a, b):
    return math.sqrt(a**2 + b**2)

euclidean_distance_pt(10, 20)

Timed at 414.232 milliseconds


22.360679774997898

In [4]:
euclidean_distance_pt(10, 20)

It was so fast that we couldn't record it. Time goes to the 18th fractional point (exa)


22.360679774997898

In [5]:
# @vectorize([uint8(uint8, uint8, uint8, boolean)])
@timeit
def simple_encode(pixel_value, field_norm, nonce, add_value):
    return (pixel_value + nonce) // field_norm if add_value else (pixel_value - nonce) // field_norm

encode = np.vectorize(simple_encode)

In [7]:
rng = default_rng()

nonce = np.uint8(17)
field_norm = np.uint8(rng.random()+1)
add_nonce = True

In [8]:
img = np.array([157, 203, 110], dtype=np.uint8)
img2 = np.uint8(rng.integers(low=0, high=255, size=3))

In [9]:
encode(img2, field_norm, nonce, add_nonce)

Timed at 1.00064 milliseconds
It was so fast that we couldn't record it. Time goes to the 18th fractional point (exa)
It was so fast that we couldn't record it. Time goes to the 18th fractional point (exa)
It was so fast that we couldn't record it. Time goes to the 18th fractional point (exa)


array([165, 213, 114], dtype=uint8)

In [10]:
@timeit
@jit(nopython=True)
def matrix_sum(matrix):
    total = 0.0
    
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            total += matrix[i, j]
            
    return total

In [11]:
matrix = np.arange(62500).reshape(250, 250)

matrix_sum(matrix)

It was so fast that we couldn't record it. Time goes to the 18th fractional point (exa)


1953093750.0

In [15]:
import numpy as np
import sys
from cupy import prof
from scipy import signal
import cupy as cp
from numba import cuda
from cupyx.scipy import sparse
from cupyx.profiler import time_range

In [13]:
@cuda.reduce
def sum_reduce(a, b):
    return a + b

@cuda.jit
def numba_l2_norm(x):
    start = cuda.grid(1)
    stride = cuda.gridsize(1)

    for i in range(start, x.shape[0], stride):
        x[i] = x[i] * x[i]

In [17]:
loops = 5
x = cp.random.random(2 ** 20)

threads_per_block = 64
blocks_per_grid = 16

with time_range('numba', 0): # prof.time_range('numba', 0):
    numba_l2_norm[blocks_per_grid, threads_per_block](x)
    output = np.sqrt(sum_reduce(x))
    print(output)

591.0258758041766


In [24]:
import sys

def get_python_version():
    version = sys.version_info
    
    return f'Working with Python version {version.major}.{version.minor}.{version.micro}'

get_python_version()

'Working with Python version 3.10.9'